In [ ]:
#this code will make cleversys output more usable, by
#eliminating data thats not useful,
#rearranging to make all data for a single arena represented in a single row,
#using partner/novel left/right info to rename data appropriately,
#and calculating percent phuddle, AKA partner pref scores

import platform
import os
import pandas as pd
import numpy as np

#define directory where the .xlsx file is
direc = '/Users/melkalliny/Documents/Lab/codeReview/Liza_Sep22/'
#define input filename
filename = 'group_event_export_summary_gayraver.xlsx'
#define output filename
filenameOut = 'ExportedFilename.csv'

#read in summary file
df = pd.read_excel(direc+filename)

df

In [ ]:
#this cell will clean up data in prep for reshaping it

#remove rows where value in first column is nan
df = df[df.iloc[:,0].notna()]

#find column names

df = df.reset_index()

#find row where column names are
for row in df.index:
    if 'Trial ID' in str(df.loc[row]):
        print(row)
        header = row
        
#get the column names

df.columns = df.loc[header]

#rename weird Events column name that has spaces

df = df.rename(columns={"              Events               ": "Events"})

#remove header (extra info) above where the data starts

df.drop(df.loc[0:header].index, inplace=True)

df

In [ ]:
#this cell will reshape data such that 
#each arena is a row, and all corresponding behavior data
#is contained within that row

#unique behaviors
unique_behs = df['Events'].unique()
# print(unique_behs)

#unique animals
animals = df['Animal id'].unique()

#find filled in columns in cleversys (those that come before "Events" column)
vid_loc = df.columns.get_loc("Video Name")
ev_loc = df.columns.get_loc("Events")+1 #need plus one to include "Events"

col_names = df.columns[vid_loc:ev_loc]
col_names = col_names.insert(-1, 'Duration(Second)')
col_names = col_names.insert(-1, 'Latency(Second)')

#columns to not pivot
col_names_nopivot = col_names[0:-3]

#simplify data down to what we need
new_df = df.loc[:, col_names]

#make wide df (rearranging all data except for latency data)
df_durations = new_df.pivot(index = col_names_nopivot, columns='Events', values = 'Duration(Second)')
df_durations = df_durations.reset_index()

#df for latency values (with some renaming) 
df_latency = new_df.pivot(index = col_names_nopivot, columns = 'Events', values = 'Latency(Second)')
df_latency = df_latency.reset_index()
df_latency = df_latency[['Animal id', 
                         'Timepoint', 
                         'Social Contact [ 1 with 2 ] in Area left while Joint Motion < 0.030', 
                         'Social Contact [ 1 with 3 ] in Area right while Joint Motion < 0.030']]
df_latency = df_latency.set_index(['Animal id', 'Timepoint']).add_prefix('latency ').reset_index()

#merge the two
df_wide = df_durations.merge(df_latency, on = ['Animal id', 'Timepoint'])

df_wide

In [ ]:
#this cell will use the physical side of the partner to assign data
#and will calculate percent phuddle, before saving output data

#find side of partner and assign huddle time
df_wide['huddle time partner'] = np.where(df_wide['Side of partner']== 'left', 
                                          df_wide['Social Contact [ 1 with 2 ] in Area left while Joint Motion < 0.030'], 
                                          df_wide['Social Contact [ 1 with 3 ] in Area right while Joint Motion < 0.030'])
df_wide['huddle time novel'] = np.where(df_wide['Side of partner']== 'left', 
                                        df_wide['Social Contact [ 1 with 3 ] in Area right while Joint Motion < 0.030'], 
                                        df_wide['Social Contact [ 1 with 2 ] in Area left while Joint Motion < 0.030'])

#total huddle
df_wide['huddle time total'] = df_wide['huddle time partner'] + df_wide['huddle time novel']

#percent phuddle
df_wide['percent phuddle'] = df_wide['huddle time partner']/(df_wide['huddle time partner'] + df_wide['huddle time novel'])
#chamber times


df_wide['chamber time partner'] = np.where(df_wide['Side of partner']== 'left', 
                                           df_wide['Area:Mouse 1 Center In left chamber'], 
                                           df_wide['Area:Mouse 1 Center In right chamber'])
df_wide['chamber time novel'] = np.where(df_wide['Side of partner']== 'left', 
                                         df_wide['Area:Mouse 1 Center In right chamber'], 
                                         df_wide['Area:Mouse 1 Center In left chamber'])
df_wide['chamber time center'] = 10800 - (df_wide['chamber time partner'] + df_wide['chamber time novel'])


df_wide['latency partner huddle'] = np.where(df_wide['Side of partner']== 'left', 
                                             df_wide['latency Social Contact [ 1 with 2 ] in Area left while Joint Motion < 0.030'], 
                                             df_wide['latency Social Contact [ 1 with 3 ] in Area right while Joint Motion < 0.030'])
df_wide['latency novel huddle'] = np.where(df_wide['Side of partner']== 'left', 
                                           df_wide['latency Social Contact [ 1 with 3 ] in Area right while Joint Motion < 0.030'], 
                                           df_wide['latency Social Contact [ 1 with 2 ] in Area left while Joint Motion < 0.030'])

#metrics to put in final csv
good_metrics = ['huddle time partner', 'huddle time novel', 'huddle time total', 'percent phuddle', 
                'chamber time partner', 'chamber time novel', 'chamber time center',
                'latency partner huddle', 'latency novel huddle']

out_cols = np.concatenate((col_names_nopivot, good_metrics))
df_out = df_wide[out_cols]

df_out

df_out.to_csv(direc+filenameOut)
